In [ ]:
#Import packages
import instaloader #Instagram package

import networkx as nx #Network packages
from pyvis.network import Network 

import os

#Initialise packages
G = nx.DiGraph()
net = Network(700, 700, directed=True,notebook = False) #For jupyter notebook = True
L = instaloader.Instaloader()

#Login details
USER = 'followers_following'
PASSWORD = 'FollowWeb!!'

# Login to instagram 
try:
    L.load_session_from_file(USER) #Use existing session (instaloader -l followers_following -p FollowWeb!!) to make new
except FileNotFoundError:
    L.login(USER, PASSWORD) #Login when session dosen't exist
    save_session_to_file()

#Open or create follow/followee file
f = open("%s.txt" % USER, "a") #append mode will create a file if none exist 
f.close() #Close file 
#Structure - Username of account /n followers /n followees

#================================= CREATE FOLLOWER AND FOLLOWEE LISTS =================================
#Check which accounts have already been scanned
recScanned = [] #List of records scanned and stored in .txt
i = 0
f = open("%s.txt" % USER, "r") #open file in read mode

filesize = os.path.getsize("%s.txt" % USER)
if (filesize > 0): #Skip if file is empty
    lines = f.readlines() #Make an array containing each line
    while (i < len(lines)):
        recScanned.append(lines[i].rstrip("\n")) #Scan in already found users and remove trailing /n
        i += 3
f.close() #Close file 

profile = instaloader.Profile.from_username(L.context, USER) # Get profile data

# Retrieve the usernames of all followers and folowees of users account
OGFollowerList = []
OGFolloweeList = []
follower = []
followee = []

print(USER)
if not(USER in recScanned):
    f = open("%s.txt" % USER, "a") # open file in append mode
    
    """ - Default Slow Method 
    for follower in profile.get_followers(): #Create a list of followers of follower 
        OGFollowerList.append(follower.username) # Add username to list 
    
    for followee in profile.get_followees(): #Create a list of followers of follower 
        OGFolloweeList.append(followee.username) # Add username to list 
    """
    
    #Custom faster version 
    followers = list(profile.get_followers())
    followees = list(profile.get_followers())
    
    for i in range (0, len(followers)):
        follower = str(followers[i]).split(" ")
        OGFollowerList.append(follower[1]) # Add username to list 
        
    for i in range (0, len(followees)):
        followee = str(followees[i]).split(" ")
        OGFolloweeList.append(followee[1]) # Add username to list 
    
    #Write to file
    f.write("%s\n" % profile.username) # Name of profile being scanned
    combFollowList = ",".join(OGFollowerList) #Combine and comma seperate follower list
    f.write("%s\n" % combFollowList) #Comma seperated follower list
    combFolloweeList = ",".join(OGFolloweeList) #Combine and comma seperate followee list
    f.write("%s\n" % combFolloweeList) #Comma seperated followee list
else:
    #read from txt into list
    f = open("%s.txt" % USER, "r") #open file in read mode
    OGfollowerList = lines[1].rstrip("\n").split(",") #Read in followers removing trailing newline and splitting by ,

f.close() #Close file 

# Retrieve the usernames of all followers and folowees of next layers of account
curFollowerList = []
curFolloweeList = []
i = 0
f = open("%s.txt" % USER, "a") # open file in append mode
while len(OGfollowerList) > 0: #for i in range (0, 25):
    if not(OGfollowerList[0] in recScanned):
        profile = instaloader.Profile.from_username(L.context, OGfollowerList[0]) # Get profile data
        print (OGfollowerList[0])

        """ - Default Slow Method 
        for follower in profile.get_followers(): #Add to list for followers of follower 
            curFollowerList.append(follower.username) # Add username to list 
        for followee in profile.get_followees(): #Create a list of followers of followee
            curFolloweeList.append(followee.username) # Add username to list 
        """
        
        #Custom faster version 
        followers = list(profile.get_followers())
        followees = list(profile.get_followers())
    
        for i in range (0, len(followers)):
            follower = str(followers[i]).split(" ")
            curFollowerList.append(follower[1]) # Add username to list 

        for i in range (0, len(followees)):
            followee = str(followees[i]).split(" ")
            curFolloweeList.append(followee[1]) # Add username to list 
        
        #Write to file
        f.write("%s\n" % profile.username) # Name of profile being scanned
        combFollowList = ",".join(curFollowerList) #Combine and comma seperate follower list
        f.write("%s\n" % combFollowList) #Comma seperated follower list
        combFolloweeList = ",".join(curFolloweeList) #Combine and comma seperate followee list
        f.write("%s\n" % combFolloweeList) #Comma seperated followee list
        
        curFollowerList = []
        curFolloweeList = []
    else:
        print (OGfollowerList[0])
    
    del OGfollowerList[0]

f.close() #Close file 

print ("Instagram scrape complete")

#=================================== CONVERT LISTS INTO NETWORKX EDGES =====================================
f = open("%s.txt" % USER, "r") #open file in read mode
lines = f.readlines() #Make an array containing each line
i = 0
y = 0
while (i < len(lines)):
        #Scan in from file
        curProfile = (lines[i].rstrip("\n")) #Scan in username
        curFollowerList = (lines[i+1].rstrip("\n").split(",")) #Scan in follower into list
        curFolloweeList = (lines[i+2].rstrip("\n").split(",")) #Scan in followee into list
        
        #Create edges
        x = 0
        addedList = []
        for x in range (0, len(curFollowerList)):
            G.add_edge(curFollowerList[x], curProfile) # Add edge to networkx
        for x in range (0, len(curFolloweeList)):
            G.add_edge(curProfile, curFolloweeList[x]) # Add edge to networkx
    
        i += 3

f.close() #Close file 

#============================================= DOWNLOAD PFPs ===============================================

numNodes = G.number_of_nodes() #Work out how many nodes
allNodes = list(G.nodes) # list all node names
for i in range (0, numNodes):
    profile = instaloader.Profile.from_username(L.context, allNodes[i]) # Get profile data
    L.download_profilepic(profile)
    print ("Downloaded ", profile.username)
print ("PFPs Downloaded")

#============================================ PROCESS NETWORK ==============================================
PrintStats = True #Print stats about network before and after processing 
MinimumNumConnections = 0 #Set the minimum number of node connections, 0 for off 
MinimumNumConnectionsAggressive = 6 #Set the minimum number of node connections, 0 for off 
DeleteAccountConnections = '' #Delete nodes connected to specified users account
DeleteAccountConnectionsExFirst = '' #Delete nodes connected to specified users account except for first ring
RemoveUser = '' #Remove a user
RemovePopular = 0 #Remove celebrity and meme accounts that dont follow more than x people back, 0 if off (should be less than min connections)

def numEdges(nodeID):
    return len(G.in_edges(nodeID)) + len(G.out_edges(nodeID))
def listEdges(nodeID):
    l1 = G.in_edges(nodeID)
    l2 = G.out_edges(nodeID)
    new = set(l2) - set(l1)
    l = list(l1 + list(new))
    return l
    
#Remove popular celebrities and meme accounts who dont follow people back 
if (RemovePopular != 0):
    popRemoved = 0
    numNodes = G.number_of_nodes() #Work out how many nodes
    allNodes = list(G.nodes) # list all node names
    for i in range (0, numNodes):
        if (len(G.out_edges(allNodes[i])) < RemovePopular): #Check if node has under x edges followings
            G.remove_node(allNodes[i]) #Remove it 
            popRemoved += 1
    print ("(RemovePopular) Number of celebs removed: ", popRemoved)

#Remove nodes with under x connection not connected to original account
deletedNodesTot = 0
if (MinimumNumConnections != 0):
    numNodes = G.number_of_nodes() #Work out how many nodes
    allNodes = list(G.nodes) # list all node names
    for i in range (0, numNodes):
        if (numEdges(allNodes[i]) <= MinimumNumConnections): #Check if node has less than 2 connections
            G.remove_node(allNodes[i]) #Remove it 
            deletedNodesTot += 1
    print ("(MinConnections) Number of nodes removed: ", deletedNodesTot)
                
#Remove nodes with under x connection not connected to original account repeats until all over the set number
deletedNodes = 1
deletedNodesTot = 0
if (MinimumNumConnectionsAggressive != 0):
    while (deletedNodes != 0): #Repeat until new nodes arent being deleted 
        deletedNodes = 0
        numNodes = G.number_of_nodes() #Work out how many nodes
        allNodes = list(G.nodes) # list all node names
        for i in range (0, numNodes):
            if (numEdges(allNodes[i]) <= MinimumNumConnectionsAggressive): #Check if node has less than 2 connections
                G.remove_node(allNodes[i]) #Remove it 
                deletedNodes += 1
                deletedNodesTot += 1
    print ("(MinConnectionsAggressive) Number of nodes removed: ", deletedNodesTot)
        
#Remove nodes connected to selected user
deletedNodesTot = 1
if (DeleteAccountConnections != ''):
    connectionsToMain = listEdges(DeleteAccountConnections) #Convert the object to a list so its subscriptale
    for i in range (0, len(connectionsToMain)):
        G.remove_node(connectionsToMain[i][1]) #Remove nodes connected to user
        deletedNodesTot += 1
    
    G.remove_node(DeleteAccountConnections) #Remove users node
    
    #Remove connectionless nodes 
    numNodes = G.number_of_nodes() #Work out how many nodes
    allNodes = list(G.nodes) # list all node names
    for i in range (0, numNodes):
        if (numEdges(allNodes[i]) == 0): #Check if node has 0 connections
            G.remove_node(allNodes[i]) #Remove it 
            deletedNodesTot += 1
    print ("(DelteAccountConnections) Number of nodes connected to ", DeleteAccountConnections, " removed: ", deletedNodesTot)
    
#Remove nodes connected to selected user that arent one of originals 
deletedNodesTot = 1
if (DeleteAccountConnectionsExFirst != ''):
    connectionsToMain = listEdges(DeleteAccountConnectionsExFirst) #Convert the object to a list so its subscriptale
    for i in range (0, len(connectionsToMain)):
        if not(connectionsToMain[i][1] in recScanned):
            G.remove_node(connectionsToMain[i][1]) #Remove nodes connected to user
            deletedNodesTot += 1
    
    G.remove_node(DeleteAccountConnectionsExFirst) #Remove users node
    
    #Remove connectionless nodes 
    numNodes = G.number_of_nodes() #Work out how many nodes
    allNodes = list(G.nodes) # list all node names
    for i in range (0, numNodes):
        if (numEdges(allNodes[i]) == 0): #Check if node has 0 connections
            G.remove_node(allNodes[i]) #Remove it 
            deletedNodesTot += 1
    print ("(DelteAccountConnectionsExFirst) Number of nodes connected to ", DeleteAccountConnectionsExFirst, " removed: ", deletedNodesTot)
    
#Remove a selected user
if (RemoveUser != ''):
    G.remove_node(RemoveUser) #Remove users node
    
    #Remove connectionless nodes 
    numNodes = G.number_of_nodes() #Work out how many nodes
    allNodes = list(G.nodes) # list all node names
    for i in range (0, numNodes):
        if (numEdges(allNodes[i]) == 0): #Check if node has 0 connections
            G.remove_node(allNodes[i]) #Remove it 
            
    print ("(RemoveUser) Removed node: ", RemoveUser)

#======================================== NETWORKX TO PYVIS =============================================
#net.from_nx(G) #Convert from networkx to pyvis

#Aesthetic Options
profilePicFill = 0 #Nodes filled with profile picture 
sizeByConnections = 1 #Change a nodes size by number of connections 
colourByConnections = 0 #Change a nodes colour by number of connections 
borderColourByConnections = 0 #Change a nodes border colour by number of connections 
edgeWidthByFollowBack = 0 #Change edge width by mutual follow 
edgeColourByFollowBack = 0 #Change edge colour by mutual follow 
shapeOnDegree = 0 #Change shape based on degree from center

#Node value functions 
def nodeSize(nodeName):
    nodeSize = 10 #Default
    if (sizeByConnections == 1):
        nodeSize = (numEdges(nodeName)/50)+9
    #if (colourByConnections == 1):
        #nodeColour = #truncate values and round to nearest whole number then pick out a colour from colour scale list
        
    return nodeSize

#Add nodes to pyvis 
numNodes = G.number_of_nodes() #Work out how many nodes
allNodes = list(G.nodes) # list all node names
usernameIndex = [] #Keep track of which ID corresponds to which username (username,ID,username,ID...)
for i in range (0, numNodes):
    net.add_node(i, allNodes[i], size = nodeSize(allNodes[i]))
    usernameIndex.append(allNodes[i])
    usernameIndex.append(i)
    
#Add edges to pyvis 
numEdges = G.number_of_edges() #Work out how many nodes
allEdges = list(G.edges) # list all node names
for i in range (0, numEdges):
    nodelistID1 = usernameIndex[usernameIndex.index(allEdges[i][0])+1] #Find ID of username edge being added 
    nodelistID2 = usernameIndex[usernameIndex.index(allEdges[i][1])+1] #by searching index returning value+1
    net.add_edge(nodelistID1, nodelistID2) 

net.force_atlas_2based(gravity=-50, central_gravity=0.01, spring_length=100, spring_strength=0.07, damping=0.8, overlap=1)
net.show_buttons(filter_=['physics'])

net.show("FollowWeb.html") #Show network
